In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dont-call-me-turkey/sample_submission.csv
/kaggle/input/dont-call-me-turkey/train.json
/kaggle/input/dont-call-me-turkey/test.json


In [2]:
from keras.models import Model
from keras.layers import Dense, Dropout, Bidirectional, CuDNNGRU, Reshape,GlobalMaxPooling1D, GlobalAveragePooling1D, Input, concatenate, BatchNormalization

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import accuracy_score

In [4]:
df_train = pd.read_json('/kaggle/input/dont-call-me-turkey/train.json')
df_test = pd.read_json('/kaggle/input/dont-call-me-turkey/test.json')

In [6]:
df_train.shape

(1195, 5)

In [7]:
max_len = 10
feature_size = 128

In [8]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [9]:
X = pad_sequences(df_train['audio_embedding'], maxlen = max_len, padding = 'post')
X_test = pad_sequences(df_test['audio_embedding'], maxlen= max_len, padding = 'post')

In [10]:
y = df_train['is_turkey'].values

In [11]:
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.2)

In [12]:
X_train.shape

(956, 10, 128)

In [16]:
def build_model():
    inp = Input(shape=(max_len, feature_size))
    x = BatchNormalization()(inp)
    x = Bidirectional(CuDNNGRU(128, return_sequences=True))(x)
    x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    concat = concatenate([avg_pool, max_pool])
    concat = Dense(64, activation='relu')(concat)
    concat = Dropout(0.5)(concat)
    output = Dense(1, activation='sigmoid')(concat)
    model = Model(inputs=inp, outputs=output)
    model.compile(loss='binary_crossentropy', optimizer = Adam(learning_rate=0.0001), metrics=['accuracy'])
    return model

In [102]:
model = build_model()

In [55]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 10, 128)]    0           []                               
                                                                                                  
 batch_normalization_4 (BatchNo  (None, 10, 128)     512         ['input_5[0][0]']                
 rmalization)                                                                                     
                                                                                                  
 bidirectional_8 (Bidirectional  (None, 10, 256)     198144      ['batch_normalization_4[0][0]']  
 )                                                                                                
                                                                                            

In [72]:
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.1, patience=2, verbose=1, min_lr = 1e-8)

In [53]:
X_train

array([[[179,  32, 240, ..., 187,   0, 255],
        [184,  38, 230, ..., 166, 188, 255],
        [179,  34, 245, ..., 166,  87, 255],
        ...,
        [186,  39, 245, ..., 255,  76, 184],
        [178,  39, 213, ...,  30, 255, 255],
        [179,  45, 200, ...,  73,  90, 255]],

       [[152,  15, 174, ...,   0,  16, 255],
        [167,  25, 187, ..., 237, 255, 255],
        [162,  17, 200, ...,  90, 171, 255],
        ...,
        [164,  32, 174, ..., 170,  78, 255],
        [166,  32, 190, ..., 131,  59, 255],
        [177,  23, 183, ...,  87,   0, 255]],

       [[191,  51, 243, ...,  41,  42, 255],
        [206,  57, 255, ...,   0, 227, 255],
        [194,  49, 246, ..., 134,  90, 255],
        ...,
        [188,  51, 235, ...,  72,  38, 255],
        [171,  40, 197, ..., 145, 156, 255],
        [177,  48, 220, ...,  96, 171, 255]],

       ...,

       [[147,   4, 159, ..., 239, 165, 255],
        [149,   5, 162, ..., 255, 199, 255],
        [152,  19, 169, ..., 154,  15, 255

In [103]:
model.fit(X_train, y_train, batch_size=256, epochs=60, validation_data=[X_val, y_val],
         callbacks=[reduce_lr], verbose=2)

Epoch 1/75
4/4 - 3s - loss: 0.6710 - accuracy: 0.5711 - val_loss: 0.7118 - val_accuracy: 0.4603 - lr: 1.0000e-04 - 3s/epoch - 849ms/step
Epoch 2/75
4/4 - 0s - loss: 0.5830 - accuracy: 0.7061 - val_loss: 0.6752 - val_accuracy: 0.5858 - lr: 1.0000e-04 - 70ms/epoch - 17ms/step
Epoch 3/75
4/4 - 0s - loss: 0.5212 - accuracy: 0.8013 - val_loss: 0.6372 - val_accuracy: 0.7071 - lr: 1.0000e-04 - 85ms/epoch - 21ms/step
Epoch 4/75
4/4 - 0s - loss: 0.4567 - accuracy: 0.8609 - val_loss: 0.5990 - val_accuracy: 0.8075 - lr: 1.0000e-04 - 81ms/epoch - 20ms/step
Epoch 5/75
4/4 - 0s - loss: 0.4042 - accuracy: 0.8912 - val_loss: 0.5610 - val_accuracy: 0.8452 - lr: 1.0000e-04 - 83ms/epoch - 21ms/step
Epoch 6/75
4/4 - 0s - loss: 0.3508 - accuracy: 0.9205 - val_loss: 0.5241 - val_accuracy: 0.8577 - lr: 1.0000e-04 - 66ms/epoch - 17ms/step
Epoch 7/75
4/4 - 0s - loss: 0.3146 - accuracy: 0.9351 - val_loss: 0.4886 - val_accuracy: 0.8577 - lr: 1.0000e-04 - 85ms/epoch - 21ms/step
Epoch 8/75
4/4 - 0s - loss: 0.2862 

In [104]:
y_test = model.predict(X_test, verbose = 1)

38/38 [==============================] - 1s 3ms/step


In [48]:
len(y_test)

1196

In [32]:
sub = pd.read_csv('/kaggle/input/dont-call-me-turkey/sample_submission.csv')

In [94]:
sub

,vid_id,is_turkey
0,pyKh38FXD3E,0.000075
1,THhP1idrWXA,0.000022
2,jsw3T6GY2Nw,0.016304
3,nFkXTMHcjMU,0.000230
4,Au8g9kAlrLQ,0.487662
...,...,...
1191,lQOQR6HCGyE,0.979828
1192,CqhPMjumTOA,0.051572
1193,Ccn2LQJc2MI,0.999747
1194,f6OhmrIdB-g,0.002077


In [105]:
sub['is_turkey'] = y_test
sub

,vid_id,is_turkey
0,pyKh38FXD3E,0.000040
1,THhP1idrWXA,0.000016
2,jsw3T6GY2Nw,0.010947
3,nFkXTMHcjMU,0.000216
4,Au8g9kAlrLQ,0.902672
...,...,...
1191,lQOQR6HCGyE,0.970870
1192,CqhPMjumTOA,0.321852
1193,Ccn2LQJc2MI,0.999848
1194,f6OhmrIdB-g,0.003648


In [106]:
sub.to_csv('submission10.csv',index=False)